In [1]:
# /Users/thomasthephasdin/Desktop/ChatGPT_API-Udemy/UPDATED-MARCH-2023-Notebooks-and-Files/01-NLP-to-SQL/data/sales_data_sample.csv
# Data we are going to use, in CSV format

In [2]:
# Import what we need, including openai package
import os
import openai

In [8]:
# setup API key as variable
openai.api_key = os.getenv('OPENAI_API_KEY')

In [9]:
# import pandas so we can interact with the csv data
import pandas as pd

In [11]:
# bring in the csv data from local folder
df = pd.read_csv("/Users/thomasthephasdin/Desktop/ChatGPT_API-Udemy/UPDATED-MARCH-2023-Notebooks-and-Files/01-NLP-to-SQL/data/sales_data_sample.csv")

In [18]:
# structure of the data
df.head(5)

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME
0,10107,30,95.70,2871.00,2/24/2003 0:00,1,2,2003,Motorcycles,2125557818,897 Long Airport Avenue,NYC,NY,10022,USA,Yu,Kwai
1,10121,34,81.35,2765.90,5/7/2003 0:00,2,5,2003,Motorcycles,26.47.1555,59 rue de l'Abbaye,Reims,NaN,51100,France,Henriot,Paul
2,10134,41,94.74,3884.34,7/1/2003 0:00,3,7,2003,Motorcycles,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,NaN,75508,France,Da Cunha,Daniel
3,10145,45,83.26,3746.70,8/25/2003 0:00,3,8,2003,Motorcycles,6265557265,78934 Hillside Dr.,Pasadena,CA,90003,USA,Young,Julie
4,10159,49,100.00,5205.27,10/10/2003 0:00,4,10,2003,Motorcycles,6505551386,7734 Strong St.,San Francisco,CA,NaN,USA,Brown,Julie


In [19]:
# example in SQL of summing up the sales by quarter
df.groupby("QTR_ID").sum()['SALES']

QTR_ID
1    2350817.73
2    2048120.30
3    1758910.81
4    3874780.01
Name: SALES, dtype: float64

In [20]:
# goal - user says "What was sales sum per quarter"
# --> like this query. SELECT SUM(SALES) FROM table WHERE .....

In [21]:
# connect to sql thru python, using sqlalchemy
import sqlalchemy

In [22]:
# create an engine in memory
from sqlalchemy import create_engine
from sqlalchemy import text

In [23]:
# Temp DB in RAM
# PUSH pandas DF --> TEMP DB
# SQL query on TEMP DB


In [24]:
# create the temp db in our RAM using create engine, echo just shows us the output
temp_db = create_engine('sqlite:///:memory:',echo=True)

In [25]:
# make the dataframe a sql database , see the SQL commands in output
data = df.to_sql(name='Sales',con=temp_db)

2023-05-04 11:54:48,755 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-05-04 11:54:48,756 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-04 11:54:48,759 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-05-04 11:54:48,759 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-04 11:54:48,762 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-04 11:54:48,763 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT
)


2023-05-04 11:54:48,764 INFO sqlalchemy.engine.Engine [no key 0.00064s] ()
2023-05-04 11:54:48,765 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Sales_index" ON "Sales" (

In [28]:
# connect to this temp db and query
with temp_db.connect() as conn:
    # makes the connection
    # run code indentation/block
    result = conn.execute(text("SELECT SUM(SALES) FROM Sales"))
    # auto close connection

2023-05-04 12:08:23,264 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM Sales
2023-05-04 12:08:23,266 INFO sqlalchemy.engine.Engine [generated in 0.00228s] ()


In [29]:
# total sum of sales for entire table
result.all()

[(10032628.85000001,)]

In [30]:
# need to describe to openai/chatgpt, what the db looks like. 
# This is how we want the prompt to look like for chatgpt

# sqlite SQL tables, with their properties
# 
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees
# SELECT

In [31]:
# function that creates this intro prompt

# we make a string of the column names , and inject it into the prompt

def create_table_definition(df):
    prompt = """### sqlite SQL table, with its properties:
    #
    # Sales({})
    #    
    """.format(",".join(str(col) for col in df.columns))
    
    return prompt

In [32]:
# prompt chatgpt to understand the database. first part of the prompt
# we are saying "there is a sales table, with these columns"
print(create_table_definition(df))

### sqlite SQL table, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #    
    


In [33]:
# function to get a natural language request 
def prompt_input():
    nlp_text = input("Enter the info you want: ")
    return nlp_text

In [54]:
# someone will ask for an query in natural language
prompt_input()

Enter the info you want: total orders in 2003


'total orders in 2003'

In [55]:
# combine the two prompts
def combine_prompts(df,query_prompt):
    definition = create_table_definition(df)
    query_init_string = f"### A query to answer: {query_prompt}\nSELECT"
    return definition+query_init_string

In [56]:
# combine that prompt with our user input 

In [93]:
# ENTER YOUR LANGUAGE QUERY HERE!! 

# the output here is what we'll be sending to chatgpt
# when you put SELECT at the end, you expect chatgpt to fill out a sql query
nlp_text = prompt_input() # NLP
combine_prompts(df,nlp_text) # DF + query that does... + NLP

Enter the info you want: list all different product lines


'### sqlite SQL table, with its properties:\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)\n    #    \n    ### A query to answer: list all different product lines\nSELECT'

In [94]:
# how the actual prompt looks
prompt = combine_prompts(df,nlp_text)
print(prompt)

### sqlite SQL table, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #    
    ### A query to answer: list all different product lines
SELECT


In [95]:
# grab the response from the openai api call
# notice we use text davinci
# also notice the stop characters to make sure gpt knows to end typing and when
response = openai.Completion.create(
        model='text-davinci-003',
        prompt = combine_prompts(df,nlp_text),
        temperature =0,
        max_tokens = 150,
        top_p = 1.0,
        frequency_penalty = 0,
        presence_penalty = 0,
        stop = ['#', ';']
)

In [96]:
# here is the chatgpt response. we need the text from it
response

<OpenAIObject text_completion id=cmpl-7CbvXGEqo8tRWQQbG2sYPn61ia2Tb at 0x7f819087ab30> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " DISTINCT PRODUCTLINE FROM Sales"
    }
  ],
  "created": 1683240083,
  "id": "cmpl-7CbvXGEqo8tRWQQbG2sYPn61ia2Tb",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 8,
    "prompt_tokens": 118,
    "total_tokens": 126
  }
}

In [97]:
# pull out just the text 
response['choices'][0]['text']

' DISTINCT PRODUCTLINE FROM Sales'

In [98]:
# concat that back 
# get rid of some white space
# add SELECT back 

def handle_response(response):
    query = response['choices'][0]['text']
    if query.startswith(" "):
        query = "SELECT"+query
    return query

In [99]:
# what the sql query looks like 
handle_response(response)

'SELECT DISTINCT PRODUCTLINE FROM Sales'

In [100]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-05-04 17:41:33,184 INFO sqlalchemy.engine.Engine SELECT DISTINCT PRODUCTLINE FROM Sales
2023-05-04 17:41:33,185 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ()


In [101]:
# final result
result.all()

[('Motorcycles',),
 ('Classic Cars',),
 ('Trucks and Buses',),
 ('Vintage Cars',),
 ('Planes',),
 ('Ships',),
 ('Trains',)]